In [5]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 7.0 MB/s eta 0:00:00


In [6]:
%pip install jiwer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 95.7 MB/s eta 0:00:00


In [ ]:
# --- RUN THIS CELL TO FIX THE ERROR ---

# 1. Force-uninstall any broken versions
print("Uninstalling old versions...")
!pip uninstall -y torch torchaudio torchdata torchvision fastai

# 2. Reinstall the correct, matching versions
print("\nInstalling fresh, matching versions...")
!pip install torch torchaudio torchvision fastai torchcodec

# 3. Restart runtime to apply changes
print("\nRestarting runtime...")
import os
os.kill(os.getpid(), 9)

Uninstalling old versions...
Found existing installation: torch 2.8.0+cu126
Uninstalling torch-2.8.0+cu126:
  Successfully uninstalled torch-2.8.0+cu126
Found existing installation: torchaudio 2.8.0+cu126
Uninstalling torchaudio-2.8.0+cu126:
  Successfully uninstalled torchaudio-2.8.0+cu126
Found existing installation: torchdata 0.11.0
Uninstalling torchdata-0.11.0:
  Successfully uninstalled torchdata-0.11.0
Found existing installation: torchvision 0.23.0+cu126
Uninstalling torchvision-0.23.0+cu126:
  Successfully uninstalled torchvision-0.23.0+cu126
Found existing installation: fastai 2.8.4
Uninstalling fastai-2.8.4:
  Successfully uninstalled fastai-2.8.4

Installing fresh, matching versions...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 899.7/899.7 MB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 594.3/594.3 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 99.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.0/88.0 

In [1]:
%pip install openai-whisper

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 803.2/803.2 kB 53.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for openai-whisper: filename=openai_whisper-20250625-py3-none-any.whl size=803979 sha256=ad0c61f6343ba6a644ac3c6e4e6d48a9bd2320e034cbcffca9012d38dc6f1031
  Stored in directory: /root/.cache/pip/wheels/61/d2/20/09ec9bef734d126cba375b15898010b6cc28578d8afdde5869
Successfully built openai-whisper


In [4]:
import pandas as pd
from openpyxl import load_workbook

excel_file = "/content/Final-Dataset--VCD.xlsx"
sheet_name = "Sheet1"

wb = load_workbook(excel_file, data_only=True)

# Print out all sheet names to help diagnose the issue
print("Available sheet names:", wb.sheetnames)

ws = wb[sheet_name]

data = []

for row in ws.iter_rows(min_row=2):
    name = row[2].value
    cell = row[5]
    # Extract hyperlink (if exists)
    if cell.hyperlink:
        link = cell.hyperlink.target
    else:
        link = cell.value

    data.append((name, link))

df = pd.DataFrame(data, columns=["audio_name", "drive_link"])
df.to_csv("dataset_links.csv", index=False)

print("✅ Extracted hyperlinks to dataset_links.csv")

Available sheet names: ['Sheet1']
✅ Extracted hyperlinks to dataset_links.csv


In [23]:
import os
import whisper
import pandas as pd
import jiwer  # Library for calculating Word Error Rate
from tqdm import tqdm  # For a nice progress bar
import subprocess # Import subprocess to check for ffmpeg


CSV_FILE_PATH = "/content/Final-Dataset-VCD.csv"

# Path to the directory containing all your audio files
AUDIO_FOLDER_PATH = "/content/downloaded_audio" # Updated to a directory path


AUDIO_FILENAME_COLUMN = "TRANSCRIPT"


TRANSCRIPTION_COLUMN = "TRANSCRIPT"


MODEL_SIZE = "medium.en"
# -----------------------------------------------------------------------------

def check_ffmpeg():
    """Checks if ffmpeg is installed and accessible."""
    try:
        subprocess.run(['ffmpeg', '-version'], capture_output=True, check=True)
        print("✅ ffmpeg is installed and accessible.")
        return True
    except (subprocess.CalledProcessError, FileNotFoundError):
        print("❌ ffmpeg is not installed or not accessible.")
        print("Please install ffmpeg. On Debian/Ubuntu, you can use: sudo apt update && sudo apt install ffmpeg")
        return False


def load_data(csv_path):
    """Loads the metadata CSV into a pandas DataFrame."""
    print(f"Loading metadata from {csv_path}...")
    try:
        df = pd.read_csv(csv_path)
    except FileNotFoundError:
        print(f"ERROR: CSV file not found at: {csv_path}")
        print("Please update the 'CSV_FILE_PATH' variable in the script.")
        return None
    except Exception as e:
        print(f"Error loading CSV: {e}")
        return None


    if AUDIO_FILENAME_COLUMN not in df.columns or TRANSCRIPTION_COLUMN not in df.columns:
        print(f"ERROR: CSV must contain '{AUDIO_FILENAME_COLUMN}' and '{TRANSCRIPTION_COLUMN}' columns.")
        print("Please update the column name variables in the script if yours are different.")
        return None

    print(f"Found {len(df)} entries in {CSV_FILE_PATH}")
    return df

def preprocess_text(text):
    """
    Applies standard text cleaning to both reference and predicted text
    to ensure a fair comparison for WER calculation.
    """
    if not isinstance(text, str):
        text = str(text)

    text = text.lower()  # Convert to lowercase
    # This is jiwer's standard transformation
    text = jiwer.RemoveMultipleSpaces()(text)
    text = jiwer.RemovePunctuation()(text)
    text = text.strip()
    return text

# Modified evaluate_dataset to search the whole directory
def evaluate_dataset(df, audio_dir, model):
    """
    Transcribes all audio files and compares them to the reference transcripts.
    Searches the entire audio directory for each file.
    """
    predictions = []
    references = []
    processed_count = 0
    skipped_files = [] # List to store names of files that failed to process

    print(f"\nStarting transcription with '{MODEL_SIZE}' model...")

    # Use tqdm for a progress bar
    for index, row in tqdm(df.iterrows(), total=df.shape[0], desc="Transcribing"):
        audio_filename = row[AUDIO_FILENAME_COLUMN]
        reference_transcript = row[TRANSCRIPTION_COLUMN]

        # Check if file_name is a valid string and not NaN
        if not isinstance(audio_filename, str) or pd.isna(audio_filename):
            skipped_files.append(f"Row {index}: Invalid file name: {audio_filename}")
            continue

        found_file_path = None
        # Search for the file in the entire audio directory structure
        for root, dirs, files in os.walk(audio_dir):
            if audio_filename in files:
                found_file_path = os.path.join(root, audio_filename)
                break # Stop searching once the file is found

        if not found_file_path:
            skipped_files.append(f"Row {index}: File not found: {audio_filename}")
            continue

        try:

            result = model.transcribe(found_file_path, fp16=False)
            predicted_transcript = result['text']

            # 3. Store cleaned results for comparison
            predictions.append(preprocess_text(predicted_transcript))
            references.append(preprocess_text(reference_transcript))
            processed_count += 1

        except Exception as e:
            skipped_files.append(f"File: {audio_filename}, Error: {e}")


    print(f"\nSuccessfully processed {processed_count} audio files.")
    if skipped_files:
        print(f"\nSkipped {len(skipped_files)} files due to errors:")
        for skipped in skipped_files:
            print(f"- {skipped}")

    return references, predictions, skipped_files # Return skipped_files too

def main():
    """Main function to run the evaluation."""

    # Check for ffmpeg
    if not check_ffmpeg():
        return

    # 1. Data Extraction (Load CSV)
    df = load_data(CSV_FILE_PATH)
    if df is None:
        return

    # 2. Audio Directory Check
    print(f"\n--- Checking Audio Directory ---")
    if not os.path.exists(AUDIO_FOLDER_PATH):
        print(f"ERROR: Audio directory not found at: {AUDIO_FOLDER_PATH}")
        print("Please ensure your audio files are uploaded to this directory.")
        return
    print(f"✅ Audio directory found: {AUDIO_FOLDER_PATH}")


    # Load Whisper Model
    try:
        print(f"\nLoading Whisper model '{MODEL_SIZE}'... (This may take a moment)")

        model = whisper.load_model(MODEL_SIZE) # Removed device parameter for now
        print(f"✅ Whisper model loaded on CPU (device parameter removed).")
    except Exception as e:
        print(f"Error loading Whisper model: {e}")
        print("Please ensure you have a valid model name and internet connection.")
        print("Also, check if ffmpeg is installed on your system.")
        return

    # 3. Run Evaluation
    # Pass the audio directory to the evaluation function
    references, predictions, skipped_files = evaluate_dataset(df, AUDIO_FOLDER_PATH, model) # Get skipped_files

    if not references or not predictions:
        print("\nNo files were successfully processed for evaluation. Exiting.")
        # Save skipped files info if any
        if skipped_files:
            skipped_df = pd.DataFrame({'skipped_file_info': skipped_files})
            skipped_df.to_csv("skipped_files.csv", index=False)
            print("✅ Saved skipped file information to skipped_files.csv")
        return


    # 4. Show Predicted vs. Actual
    print("\n--- Transcription Results (Actual vs. Predicted) ---")
    # We'll just show the first 10 results to avoid flooding the console
    for i in range(min(len(references), 10)):
        print(f"\nSample {i+1}:")
        print(f"  Actual:   {references[i]}")
        print(f"  Predicted: {predictions[i]}")
    if len(references) > 10:
        print(f"\n... (and {len(references) - 10} more results) ...")

    # 5. Calculate and Show WER
    print("\n--- Overall Performance ---")
    try:
        # Calculate Word Error Rate
        wer = jiwer.wer(references, predictions)
        print(f"Overall Word Error Rate (WER): {wer * 100:.2f} %")
        print(f"This means, on average, {wer*100:.2f}% of the words were incorrect.")

        # Save actual vs predicted to a CSV file
        results_df = pd.DataFrame({'actual': references, 'predicted': predictions})
        results_df.to_csv("actualvspredicted.csv", index=False)
        print("✅ Saved actual vs predicted transcripts to actualvspredicted.csv")

    except ValueError as e:
        print(f"Error calculating WER or saving results: {e}")

    # Optional: Save skipped files info even if some files were processed
    if skipped_files:
        skipped_df = pd.DataFrame({'skipped_file_info': skipped_files})
        skipped_df.to_csv("skipped_files.csv", index=False)
        print("✅ Saved skipped file information to skipped_files.csv")


if __name__ == "__main__":
    main()

✅ ffmpeg is installed and accessible.
Loading metadata from /content/Final-Dataset-VCD.csv...
Found 1536 entries in /content/Final-Dataset-VCD.csv

--- Checking Audio Directory ---
✅ Audio directory found: /content/downloaded_audio

Loading Whisper model 'medium.en'... (This may take a moment)


100%|█████████████████████████████████████| 1.42G/1.42G [00:28<00:00, 54.2MiB/s]


✅ Whisper model loaded on CPU (device parameter removed).

Starting transcription with 'medium.en' model...


Transcribing: 100%|██████████| 1536/1536 [16:58<00:00,  1.51it/s]


Successfully processed 1236 audio files.

Skipped 300 files due to errors:
- File: The battery is low, land the drone., Error: Failed to load audio: ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr 